In [1]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

Language Translation with `nn.Transformer` and torchtext
========================================================

This tutorial shows:

:   -   How to train a translation model from scratch using Transformer.
    -   Use torchtext library to access
        [Multi30k](http://www.statmt.org/wmt16/multimodal-task.html#task1)
        dataset to train a German to English translation model.

本教程展示了以下内容：

- 如何使用 Transformer 从头开始训练翻译模型。
- 使用 torchtext 库访问 [Multi30k](http://www.statmt.org/wmt16/multimodal-task.html#task1) 数据集来训练一个德语到英语的翻译模型。

Data Sourcing and Processing 数据获取和处理
============================

[torchtext library](https://pytorch.org/text/stable/) has utilities for
creating datasets that can be easily iterated through for the purposes
of creating a language translation model. In this example, we show how
to use torchtext\'s inbuilt datasets, tokenize a raw text sentence,
build vocabulary, and numericalize tokens into tensor. We will use
[Multi30k dataset from torchtext
library](https://pytorch.org/text/stable/datasets.html#multi30k) that
yields a pair of source-target raw sentences.

To access torchtext datasets, please install torchdata following
instructions at <https://github.com/pytorch/data>.

[torchtext 库](https://pytorch.org/text/stable/) 提供了创建数据集的实用工具，这些数据集可以轻松地进行迭代，以创建语言翻译模型。在这个例子中，我们展示了如何使用 torchtext 内置的数据集，对原始文本句子进行分词，构建词汇表，并将标记数值化为张量。我们将使用 [torchtext 库中的 Multi30k 数据集](https://pytorch.org/text/stable/datasets.html#multi30k)，该数据集生成一对源语言和目标语言的原始句子。

要访问 torchtext 数据集，请按照 [https://github.com/pytorch/data](https://github.com/pytorch/data) 上的说明安装 torchdata。

In [2]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import multi30k, Multi30k
from typing import Iterable, List


# We need to modify the URLs for the dataset since the links to the original dataset are broken
# Refer to https://github.com/pytorch/text/issues/1756#issuecomment-1163664163 for more info
# 我们需要修改数据集的URL，因为原始数据集的链接已损坏
multi30k.URL["train"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/training.tar.gz"
multi30k.URL["valid"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/validation.tar.gz"

SRC_LANGUAGE = 'de'
# 源语言为德语
TGT_LANGUAGE = 'en'
# 目标语言为英语

# Place-holders
# 占位符
token_transform = {}
vocab_transform = {}

/data/xuruiyuan/miniconda3/lib/python3.12/site-packages/torchtext/data/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/data/xuruiyuan/miniconda3/lib/python3.12/site-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/data/xuruiyuan/miniconda3/lib/python3.12/site-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABO

Create source and target language tokenizer. Make sure to install the
dependencies.

创建源语言和目标语言的分词器。请确保安装依赖项。

``` {.sourceCode .python}
pip install -U torchdata
pip install -U spacy
python -m spacy download en_core_web_sm
python -m spacy download de_core_news_sm
```

In [3]:
# 使用 spaCy 库为源语言（德语）和目标语言（英语）分别创建分词器
token_transform[SRC_LANGUAGE] = get_tokenizer('spacy', language='de_core_news_sm')
# 为源语言（SRC_LANGUAGE，德语）创建一个分词器，并将其存储在 token_transform 字典中对应 SRC_LANGUAGE 的位置。它使用了 spaCy 库的 de_core_news_sm 模型，这是一个适用于德语的小型语言模型
#   get_tokenizer('spacy', language='de_core_news_sm')：调用 get_tokenizer 函数，指定使用 spaCy 分词器，并选择 de_core_news_sm 模型。
#   token_transform[SRC_LANGUAGE]：将生成的德语分词器存储在 token_transform 字典中，以便后续使用。

token_transform[TGT_LANGUAGE] = get_tokenizer('spacy', language='en_core_web_sm')
# 为目标语言（TGT_LANGUAGE，英语）创建一个分词器，并将其存储在 token_transform 字典中对应 TGT_LANGUAGE 的位置。它使用了 spaCy 库的 en_core_web_sm 模型，这是一个适用于英语的小型语言模型
#   get_tokenizer('spacy', language='en_core_web_sm')：调用 get_tokenizer 函数，指定使用 spaCy 分词器，并选择 en_core_web_sm 模型。
#   token_transform[TGT_LANGUAGE]：将生成的英语分词器存储在 token_transform 字典中，以便后续使用



# helper function to yield list of tokens
def yield_tokens(data_iter: Iterable, language: str) -> List[str]:
    '''
    data_iter: 一个可迭代对象（例如列表、生成器），其中包含了数据样本
    language: 指定要处理的语言，应该是 SRC_LANGUAGE（源语言）或 TGT_LANGUAGE（目标语言）中的一个
    '''
    
    language_index = {SRC_LANGUAGE: 0, TGT_LANGUAGE: 1}
    # 创建一个字典 language_index，将源语言（SRC_LANGUAGE）映射到索引 0，目标语言（TGT_LANGUAGE）映射到索引 1

    # 假设数据样本 data_sample 是一个包含两个元素的列表或元组，第一个元素是源语言文本，第二个元素是目标语言文本
    for data_sample in data_iter:
        # 遍历 data_iter 中的每一个数据样本
        yield token_transform[language](data_sample[language_index[language]])
        '''
        1. 获取语言索引: language_index[language]
            language_index 是一个字典，它将源语言 (SRC_LANGUAGE) 映射到索引 0，将目标语言 (TGT_LANGUAGE) 映射到索引 1。
            language 是传递给函数的参数，表示要处理的语言。
            language_index[language] 根据 language 获取相应的索引值。例如，如果 language 是 SRC_LANGUAGE，那么 language_index[language] 会返回 0
        2. 获取数据样本中的特定语言文本: data_sample[language_index[language]]
            data_sample 是迭代过程中从 data_iter 中获取的每一个数据样本。
            每个 data_sample 应该是一个包含两个元素的列表或元组，第一个元素是源语言文本，第二个元素是目标语言文本。
            data_sample[language_index[language]] 使用上一步获取的索引值从 data_sample 中提取对应的文本。例如，如果索引值是 0，则提取源语言文本
        3. 分词操作 token_transform[language](data_sample[language_index[language]])
            token_transform 是一个包含分词器的字典，键是语言（SRC_LANGUAGE 或 TGT_LANGUAGE），值是相应语言的分词器。
            token_transform[language] 根据 language 获取对应语言的分词器。例如，如果 language 是 SRC_LANGUAGE，则获取源语言的分词器。
            token_transform[language](data_sample[language_index[language]]) 调用获取到的分词器对提取的文本进行分词操作，返回分词后的 token （tokens）。
        4. 生成tokens: yield token_transform[language](data_sample[language_index[language]])
            yield 是 Python 中的一个关键字，用于生成器函数。
            使用 yield 关键字将分词后的 token 生成出来。
            每次调用生成器函数时，它会暂停并返回一个 token ，下次调用时会从暂停的地方继续执行。
        示例：
            假设：
                SRC_LANGUAGE = 'de'
                TGT_LANGUAGE = 'en'
                language = 'de'（即源语言）
                data_sample = ['Das ist ein Beispiel.', 'This is an example.']
                token_transform 字典中包含针对德语和英语的分词器函数。
            执行这行代码时的步骤如下：
                language_index[language] 返回 0（因为 language 是 de）。
                data_sample[0] 返回 'Das ist ein Beispiel.'（源语言文本）。
                token_transform['de'] 获取德语的分词器。
                token_transform['de']('Das ist ein Beispiel.') 对 'Das ist ein Beispiel.' 进行分词，例如返回 ['Das', 'ist', 'ein', 'Beispiel', '.']。
                yield ['Das', 'ist', 'ein', 'Beispiel', '.'] 将这些 token 生成出来
        '''

# Define special symbols and indices
# 定义特殊符号和索引
'''
index：
    单数形式，指一个单一的位置或指针。
    例子：在数组中，index 3 指的是数组的第四个元素（因为大多数编程语言的索引是从0开始的）。
indices：
    复数形式，指多个位置或指针。
    例子：在数组操作中，如果你需要访问多个元素，可以使用indices 3, 5, 7来指代这些位置上的元素。
简而言之，index是单数形式，指一个位置，而indices是复数形式，指多个位置。
'''
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
'''
UNK_IDX (Unknown Index):
    UNK_IDX 是 "Unknown" 的缩写，用于表示在词汇表（vocabulary）中未出现的词汇的索引。任何未在训练数据中见过的词汇在模型中都会被标记为未知词。
    在序列处理中，当遇到不在词汇表中的词时，会使用这个索引来替代。
PAD_IDX (Padding Index):
    PAD_IDX 是 "Padding" 的缩写，用于表示填充符的索引。填充符用于将序列对齐到相同的长度。
    在处理不同长度的输入序列时，通常需要将序列填充到相同的长度以便进行批处理（batch processing）。填充符可以帮助对齐序列而不影响模型的学习。
BOS_IDX (Beginning of Sequence Index):
    BOS_IDX 是 "Beginning of Sequence" 的缩写，用于表示序列开始的符号的索引。
    在序列生成任务（如机器翻译、文本生成等）中，生成的序列通常以这个符号开始，提示模型这是一个新的序列的开始。
EOS_IDX (End of Sequence Index):
    EOS_IDX 是 "End of Sequence" 的缩写，用于表示序列结束的符号的索引
    在序列生成任务中，生成的序列通常以这个符号结束，提示模型序列已经结束
    
示例
    假设有一个句子 "Hello world"：
        如果 "Hello" 不在词汇表中，它会被替换为 UNK_IDX，即 0。
        如果需要对多个句子进行批处理，而这些句子的长度不同，则较短的句子会被填充，使它们的长度与最长的句子相同，填充符使用 PAD_IDX，即 1。
        在生成句子时，句子通常以 BOS_IDX 开始，即 2，提示这是一个新句子的开始。
        句子以 EOS_IDX 结束，即 3，提示这是句子的结束
    # Example sentence with special tokens
    sentence = [BOS_IDX] + [word_to_index[word] if word in vocab else UNK_IDX for word in "Hello world".split()] + [EOS_IDX]
    # Assuming 'Hello' is not in vocab and 'world' is in vocab with index 5
    # Output: sentence = [2, 0, 5, 3]

以上设置为模型处理序列数据提供了标准化的方法，使得处理不同长度的句子和未见词汇变得更加容易和一致
'''
# Make sure the tokens are in order of their indices to properly insert them in vocab
# 确保标记按照其索引的顺序排列，以便正确地插入词汇表中
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']
# special_symbols 是一个包含特殊符号的列表。每个符号在自然语言处理任务中都有特定的用途。
# '<unk>': 表示未知符号，用于替代词汇表中未出现的词汇。对应索引 UNK_IDX。
# '<pad>': 表示填充符，用于对齐不同长度的序列。对应索引 PAD_IDX。
# '<bos>': 表示序列开始符，用于标记序列的开始。对应索引 BOS_IDX。
# '<eos>': 表示序列结束符，用于标记序列的结束。对应索引 EOS_IDX。

# 使用 torchtext 库从训练数据迭代器构建词汇表。它针对源语言和目标语言分别创建了词汇表，并包含了特殊符号
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    # 遍历源语言和目标语言，SRC_LANGUAGE 和 TGT_LANGUAGE 分别表示源语言和目标语言
    
    # Training data Iterator
    train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    # Multi30k 是一个用于机器翻译任务的标准数据集。
    # split='train' 表示使用训练数据集。
    # language_pair=(SRC_LANGUAGE, TGT_LANGUAGE) 指定数据集中的语言对。
    
    # Create torchtext's Vocab object
    # 使用 torchtext 库中的 build_vocab_from_iterator 函数，从一个 token 生成器中构建词汇表（vocabulary）。具体来说，它将训练数据中的 token 提取出来，并根据这些 token 构建一个包含特殊符号的词汇表。
    vocab_transform[ln] = build_vocab_from_iterator(yield_tokens(train_iter, ln),
                                                    min_freq=1,
                                                    specials=special_symbols,
                                                    special_first=True)
    '''
    1. 获取token生成器: yield_tokens(train_iter, ln)
        调用 yield_tokens 函数，从训练数据迭代器 train_iter 中生成指定语言 ln 的token。
        train_iter 是一个迭代器，包含了训练数据样本。
        ln 表示当前处理的语言（例如 SRC_LANGUAGE 或 TGT_LANGUAGE）。
        yield_tokens(train_iter, ln) 会生成一个token迭代器，逐个生成训练数据中的token。
    2. 构建词汇表: build_vocab_from_iterator(...)
        使用 torchtext.vocab 中的 build_vocab_from_iterator 函数，从token迭代器中构建词汇表。
        传入的参数：
            yield_tokens(train_iter, ln)： token 生成器，提供从训练数据中生成的 token 。
            min_freq=1：指定词汇的最小出现频率，出现频率低于 1 的词汇将不会被包括在词汇表中。这里 1 表示所有出现过的词汇都会被包括在内。
            specials=special_symbols：包含特殊符号（<unk>, <pad>, <bos>, <eos>）。
            special_first=True：确保特殊符号出现在词汇表的最前面。
    3. 将词汇表存储在字典中: vocab_transform[ln] = build_vocab_from_iterator(...)
        vocab_transform 是一个字典，键是语言（例如 SRC_LANGUAGE 或 TGT_LANGUAGE），值是构建的词汇表。
        vocab_transform[ln] 将构建的词汇表存储在字典中对应的语言键下。
    '''

# Set ``UNK_IDX`` as the default index. This index is returned when the token is not found.
# If not set, it throws ``RuntimeError`` when the queried token is not found in the Vocabulary.
# 将 UNK_IDX 设置为默认索引。当找不到标记时，将返回此索引。
# 如果未设置，当查询的标记在词汇表中找不到时，会抛出 RuntimeError。
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    # 遍历源语言 (SRC_LANGUAGE) 和目标语言 (TGT_LANGUAGE)。
    # ln 表示当前处理的语言
    vocab_transform[ln].set_default_index(UNK_IDX)
    # vocab_transform 是一个字典，键是语言，值是对应语言的词汇表对象。
    # vocab_transform[ln] 获取当前语言的词汇表对象。
    # set_default_index(UNK_IDX) 设置默认索引，当查找一个不存在于词汇表中的 token 时，返回 UNK_IDX（即 <unk> 的索引）

In [4]:
print(vocab_transform[SRC_LANGUAGE]['<unk>'])  # Should output 0
print(vocab_transform[SRC_LANGUAGE]['<pad>'])  # Should output 1
print(vocab_transform[SRC_LANGUAGE]['<bos>'])  # Should output 2
print(vocab_transform[SRC_LANGUAGE]['<eos>'])  # Should output 3
print(vocab_transform[TGT_LANGUAGE]['<unk>'])  # Should output 0
print(vocab_transform[TGT_LANGUAGE]['<pad>'])  # Should output 1
print(vocab_transform[TGT_LANGUAGE]['<bos>'])  # Should output 2
print(vocab_transform[TGT_LANGUAGE]['<eos>'])  # Should output 3


0
1
2
3
0
1
2
3


Seq2Seq Network using Transformer 使用 Transformer 的 Seq2Seq 网络
=================================

Transformer is a Seq2Seq model introduced in ["Attention is all you
need"](https://papers.nips.cc/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf)
paper for solving machine translation tasks. Below, we will create a
Seq2Seq network that uses Transformer. The network consists of three
parts. First part is the embedding layer. This layer converts tensor of
input indices into corresponding tensor of input embeddings. These
embedding are further augmented with positional encodings to provide
position information of input tokens to the model. The second part is
the actual
[Transformer](https://pytorch.org/docs/stable/generated/torch.nn.Transformer.html)
model. Finally, the output of the Transformer model is passed through
linear layer that gives unnormalized probabilities for each token in the
target language.

Transformer 是一种 Seq2Seq 模型，在 ["Attention is all you need"](https://papers.nips.cc/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf) 论文中提出，用于解决机器翻译任务。下面，我们将创建一个使用 Transformer 的 Seq2Seq 网络。该网络由三个部分组成。第一部分是嵌入层。这个层将输入索引的张量转换为对应的输入嵌入张量。这些嵌入进一步通过位置编码增强，为模型提供输入标记的位置信息。第二部分是实际的 [Transformer](https://pytorch.org/docs/stable/generated/torch.nn.Transformer.html) 模型。最后，Transformer 模型的输出通过线性层，得到目标语言中每个标记的未归一化概率。


In [5]:

from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
# 辅助模块：将位置编码添加到标记嵌入中，以引入词序的概念
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        # emb_size: 词嵌入的维度
        # emb_size = 512
        super(PositionalEncoding, self).__init__()
        '''
        super(PositionalEncoding, self).__init__():
            这是调用父类（nn.Module）的构造函数的方法。在 Python 3 中，可以简化为 super().__init__()，效果是一样的。
            super(PositionalEncoding, self) 返回的是一个代理对象，这个代理对象将方法和属性查找委托给 PositionalEncoding 类的父类（即 nn.Module）。通过这个代理对象，我们可以调用父类的方法和访问父类的属性，而不必直接引用父类的名称。这在实现继承时非常有用，尤其是在需要确保父类的初始化逻辑被执行的情况下
            __init__() 调用父类 nn.Module 的构造函数，确保父类的初始化代码被执行。
        在 Python 中，当你定义一个类继承自另一个类时，子类需要调用父类的构造函数来初始化父类的部分。这对于继承自 nn.Module 的 PyTorch 模型类尤为重要，因为 nn.Module 包含了一些必要的初始化逻辑，例如注册模块、参数和缓冲区
        为什么使用 super():
            确保父类 nn.Module 的初始化代码被执行。
            这对于 nn.Module 类尤为重要，因为它包含了模块注册和参数初始化的逻辑。
            使用 super() 可以使代码更具可读性和可维护性，特别是在多重继承的情况下
        '''
        # print("emb_size: ", emb_size)
        # print("dropout:", dropout)
        # print("maxlen:", maxlen)
        
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        # print(math.log(10000) / emb_size)
        # print(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        # print(den)
        # print(den.shape)
        
        # den: (emb_size/2)= (256)
        '''
        torch.arange(0, emb_size, 2): 
            生成一个从 0 到 emb_size（不包括 emb_size）的序列，步长为 2。
            如果 emb_size 是 512，那么这行代码会生成一个包含 256 个元素的张量 [0, 2, 4, ..., 510]。
            这是因为位置编码中，每个位置的正弦和余弦函数的频率会交替应用，因此只需要一半的维度来计算
        math.log(10000) / emb_size:
            计算 10000 的自然对数，并将其除以嵌入维度 emb_size。
            这个值是一个常数，用于缩放频率因子，使得频率的增长速度适应嵌入维度。
            具体来说，对于 emb_size = 512，计算结果是 math.log(10000) / 512
        - torch.arange(0, emb_size, 2) * (math.log(10000) / emb_size)
            将步长为 2 的序列与缩放常数相乘，并取负值
            结果是一个线性递增的负数序列，用于生成频率因子
        torch.exp(...)
            对上述负数序列应用指数函数。
            指数函数的作用是将线性递增的负数序列转换为一个在 [0, 1] 之间变化的频率因子。
            结果是一个张量，其中每个元素代表一个特定维度的位置编码频率因子
        
        具体例子
            假设 emb_size = 512，计算过程如下：
            torch.arange(0, 512, 2) 生成一个张量 [0, 2, 4, ..., 510]。
            math.log(10000) / 512 计算常数值，大约等于 0.017988946039015984。
            负数序列计算：[0, 2, 4, ..., 510] * -0.017988946039015984，得到 [0.0000, -0.0360, -0.0720, -0.1079, ...]。
            应用指数函数 torch.exp([0.0000, -0.0360, -0.0720, -0.1079, ...])，得到 [1.0000e+00, 9.6466e-01, 9.3057e-01, 8.9769e-01, ..., 1.0366e-04]。
            最终得到的 den 张量用于生成位置编码中的频率因子，这些频率因子将与位置索引结合，以生成每个位置的正弦和余弦值，用于位置编码
        '''
        
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        # pos: (maxlen, 1)= (5000, 1) <- (5000)
        '''
        torch.arange(0, maxlen):
            生成一个从 0 到 maxlen（不包括 maxlen）的序列。
            如果 maxlen 是 5000，那么这行代码会生成一个包含 5000 个元素的张量 [0, 1, 2, ..., 4999]。
        .reshape(maxlen, 1):
            将上述生成的序列重塑为一个形状为 (maxlen, 1) 的张量。
            这样做是为了将位置编码的频率因子与位置索引对应起来。
        '''
        
        pos_embedding = torch.zeros((maxlen, emb_size))
        # pos_embedding: (maxlen, emb_size)= (5000, 512)
        
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        # pos * den: (maxlen, emb_size/2)= (5000, 256)
        # torch.sin(pos * den): (maxlen, emb_size/2)= (5000, 256)
        # pos_embedding[:, 0::2] = torch.sin(pos * den): (maxlen, emb_size/2)= (5000, 256)
        '''
        pos_embedding[:, 0::2]:
            这部分代码选择 pos_embedding 矩阵的所有行（通过 : 表示），以及列索引为偶数的位置（通过 0::2 表示）。
            0::2 是 Python 切片语法，表示从第 0 列开始，每隔 2 列选择一次，即选择所有偶数列。
        torch.sin(pos * den):
            pos 是一个形状为 (maxlen, 1) 的张量，包含位置索引。
            den 是一个形状为 (emb_size // 2,) 的张量，包含频率因子。
            pos * den 计算位置索引与频率因子的乘积。由于广播机制（broadcasting），这会生成一个形状为 (maxlen, emb_size // 2) 的张量。
            torch.sin(pos * den) 计算上述乘积的正弦值，结果是一个形状为 (maxlen, emb_size // 2) 的张量
        举例:
            pos = tensor([[0],
                          [1],
                          [2],
                          [3],
                          [4]])
            den = tensor([1.0000, 0.1778, 0.0316])
            pos_embedding[:, 0::2] = torch.sin(pos * den)
            pos_embedding = tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
                                    [ 0.8415,  0.0000,  0.1769,  0.0000,  0.0316,  0.0000],
                                    [ 0.9093,  0.0000,  0.3482,  0.0000,  0.0632,  0.0000],
                                    [ 0.1411,  0.0000,  0.5085,  0.0000,  0.0947,  0.0000],
                                    [-0.7568,  0.0000,  0.6527,  0.0000,  0.1261,  0.0000]])
        pos * den的广播操作说明示例:
        
        假设 maxlen = 5 和 emb_size = 6，则：
        pos 的形状是 (5, 1)，内容为：
            tensor([[0],
                    [1],
                    [2],
                    [3],
                    [4]])
        den 的形状是 (3,)，内容为：
            tensor([1.0000, 0.1778, 0.0316])
                
        当我们计算 pos * den 时：
            pos 的形状是 (5, 1)。
            den 的形状是 (3,)。
            
        广播机制会进行如下操作：
        首先，将 den 的形状从 (3,) 扩展为 (1, 3)。扩展后的 den 为：
            tensor([[1.0000, 0.1778, 0.0316]])
        其次，将 pos 的形状从 (5, 1) 扩展为 (5, 3)，每个元素沿列方向复制。扩展后的 pos 为：
            tensor([[0, 0, 0],
                    [1, 1, 1],
                    [2, 2, 2],
                    [3, 3, 3],
                    [4, 4, 4]])
        最后，进行元素级相乘，结果为：
            tensor([[0.0000, 0.0000, 0.0000],
                    [1.0000, 0.1778, 0.0316],
                    [2.0000, 0.3556, 0.0632],
                    [3.0000, 0.5334, 0.0948],
                    [4.0000, 0.7112, 0.1264]])
        '''
        
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        # pos * den: (maxlen, emb_size/2)= (5000, 256)
        # torch.cos(pos * den): (maxlen, emb_size/2)= (5000, 256)
        # pos_embedding[:, 1::2] = torch.cos(pos * den): (maxlen, emb_size/2)= (5000, 256)
        # pos_embedding: (maxlen, emb_size)= (5000, 512)
        '''
        pos_embedding[:, 1::2]:
            这部分代码选择 pos_embedding 矩阵的所有行（通过 : 表示），以及列索引为奇数的位置（通过 1::2 表示）。
            1::2 是 Python 切片语法，表示从第 1 列开始，每隔 2 列选择一次，即选择所有奇数列。
        torch.cos(pos * den):
            pos 是一个形状为 (maxlen, 1) 的张量，包含位置索引。
            den 是一个形状为 (emb_size // 2,) 的张量，包含频率因子。
            pos * den 计算位置索引与频率因子的乘积。由于广播机制（broadcasting），这会生成一个形状为 (maxlen, emb_size // 2) 的张量。
            torch.cos(pos * den) 计算上述乘积的余弦值，结果是一个形状为 (maxlen, emb_size // 2) 的张量
        举例:
            pos = tensor([[0],
                          [1],
                          [2],
                          [3],
                          [4]])
            den = tensor([1.0000, 0.1778, 0.0316])
            pos * den = tensor([[0.0000, 0.0000, 0.0000],
                                [1.0000, 0.1778, 0.0316],
                                [2.0000, 0.3556, 0.0632],
                                [3.0000, 0.5334, 0.0948],
                                [4.0000, 0.7112, 0.1264]])
            pos_embedding[:, 0::2] = torch.sin(pos * den)
            pos_embedding[:, 1::2] = torch.cos(pos * den)
            pos_embedding = tensor([[ 0.0000,  1.0000,  0.0000,  1.0000,  0.0000,  1.0000],
                                    [ 0.8415,  0.5403,  0.1769,  0.9842,  0.0316,  0.9995],
                                    [ 0.9093, -0.4161,  0.3482,  0.9374,  0.0632,  0.9980],
                                    [ 0.1411, -0.9900,  0.5085,  0.8611,  0.0947,  0.9955],
                                    [-0.7568, -0.6536,  0.6527,  0.7576,  0.1261,  0.9920]])
        '''
        pos_embedding = pos_embedding.unsqueeze(-2)
        # pos_embedding: (maxlen, 1, emb_size)= (5000, 1, 512)
        '''
        unsqueeze 方法:
            unsqueeze(dim) 方法在指定维度 dim 处添加一个新的维度，新的维度大小为 1。
            dim 参数可以是正数或负数。如果是负数，它表示从张量末尾的倒数第几个维度。
        -2 维度:
            这里使用 -2 作为维度参数，表示在倒数第二个维度添加一个新的维度。
            例如，对于形状为 (maxlen, emb_size) 的张量，在 -2 维度添加新维度后，结果形状为 (maxlen, 1, emb_size)。
        
        pos_embedding = tensor([[[ 0.0000,  1.0000,  0.0000,  1.0000,  0.0000,  1.0000]],
                                [[ 0.8415,  0.5403,  0.1769,  0.9842,  0.0316,  0.9995]],
                                [[ 0.9093, -0.4161,  0.3482,  0.9374,  0.0632,  0.9980]],
                                [[ 0.1411, -0.9900,  0.5085,  0.8611,  0.0947,  0.9955]],
                                [[-0.7568, -0.6536,  0.6527,  0.7576,  0.1261,  0.9920]]])
        (5,1,6)
        '''

        self.dropout = nn.Dropout(dropout)
        # dropout = 0.1
        # 如果 dropout 设置为 0.1，那么每个元素被置零的概率是 10%
        # nn.Dropout(dropout): 丢弃概率为 dropout 的 Dropout 层
        
        # 将 pos_embedding 注册为 PositionalEncoding 类的一个缓冲区（buffer）
        self.register_buffer('pos_embedding', pos_embedding)
        '''
        register_buffer 方法:
            register_buffer 是 nn.Module 类中的一个方法，用于将张量注册为模块的缓冲区。
            缓冲区是指不作为模型参数参与训练，但需要在前向传播和保存/加载模型时使用的张量。
        'pos_embedding' 参数:
            这是缓冲区的名称，类型为字符串。在模型中可以通过这个名称访问该缓冲区。
            在这个例子中，缓冲区名称为 'pos_embedding'
        pos_embedding 参数:
            这是要注册为缓冲区的张量。
            在这个例子中，pos_embedding 是计算好的位置编码张量
        为什么使用 register_buffer
            不参与参数更新: 缓冲区张量不会作为模型参数参与梯度计算和更新。
            保存和加载: 缓冲区张量会在模型保存和加载时保留，这对位置编码等固定值非常有用。
            设备管理: 缓冲区张量会随着模型移动到不同设备（如从 CPU 到 GPU）而自动移动。
        '''

    def forward(self, token_embedding: Tensor):
        # print("token_embedding shape", token_embedding.shape)
        # token_embedding: (seq_len, batch_size, emb_size)= (seq_len, 128, 512)
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])
        # self.pos_embedding: (maxlen, 1, emb_size)= (5000, 1, 512)
        # token_embedding.size(0) = seq_len
        # self.pos_embedding[:token_embedding.size(0), :]: (seq_len, 1, emb_size)= (seq_len, 1, 512)
        '''
        切片操作: self.pos_embedding[:token_embedding.size(0), :]
            self.pos_embedding 是在初始化中注册为缓冲区的张量，包含预计算的正弦和余弦位置编码。
            token_embedding.size(0) 返回输入张量的序列长度。
            self.pos_embedding[:token_embedding.size(0), :] 通过切片操作提取与输入序列长度相匹配的部分位置编码。
        位置编码和 token 嵌入相加: token_embedding + self.pos_embedding[:token_embedding.size(0), :]
            将提取出的适当长度的 pos_embedding 加到 token_embedding 上
            这种加法操作在每个位置上对嵌入向量和位置编码向量进行逐元素相加
        应用 Dropout:
            将上述相加结果应用 dropout 操作
            Dropout 随机将部分元素置零，以防止过拟合，提高模型的泛化能力
        返回值:
            返回添加了位置编码的词嵌入张量
        '''

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
# 辅助模块：将输入索引的张量转换为对应的标记嵌入张量
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        '''
        vocab_size：词汇表的大小，即模型可以处理的不同单词的数量。
        emb_size：嵌入维度，即每个单词被嵌入到多大的向量空间。
        nn.Embedding(vocab_size, emb_size)：初始化一个嵌入层。这个层将每个单词索引映射到一个大小为 emb_size 的嵌入向量。
        self.emb_size：存储嵌入维度以备后用。
        '''
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size
        # 512

    def forward(self, tokens: Tensor):
        # print("token shape", tokens.shape)
        # tokens: (seq_len, batch_size)= (seq_len, 128)
        '''
        tokens：输入的张量，包含单词的索引，形状通常为 (sequence_length, batch_size) 或 (batch_size, sequence_length)。
        self.embedding(tokens.long())：将输入的单词索引映射到嵌入向量。
            tokens.long()：确保输入的索引是 long 类型，这是 nn.Embedding 层要求的输入类型。
            输出张量形状为 (sequence_length, batch_size, emb_size)，即每个单词索引被映射到一个大小为 emb_size 的向量。
        * math.sqrt(self.emb_size)：将嵌入向量乘以 emb_size 的平方根进行缩放。
            这种缩放可以确保在位置编码和嵌入向量相加时，二者具有相似的数值范围，有助于模型的稳定训练。
        '''
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)
        # (seq_len, batch_size, emb_size)= (seq_len, 128, 512)
        # tokens.long(): (seq_len, batch_size)= (seq_len, 128)
        # self.embedding(tokens.long()): (seq_len, batch_size, emb_size)= (seq_len, 128, 512)
        # self.embedding(tokens.long()) * math.sqrt(self.emb_size): (seq_len, batch_size, emb_size)= (seq_len, 128, 512)
        '''
        nn.Embedding 接受输入张量 tokens，并将每个单词索引映射到对应的嵌入向量
        嵌入矩阵:
            nn.Embedding 层内部维护一个形状为 (vocab_size, emb_size) 的嵌入矩阵 W。
            每一行对应一个词汇表中的单词，每一列对应一个嵌入维度。
        索引查找:
            当 self.embedding(tokens.long()) 被调用时，它会根据输入张量 tokens 中的索引查找嵌入矩阵中的相应行。
        结果张量:
            对于输入张量 tokens 中的每个索引，查找嵌入矩阵 W 中的对应行，并将这些行组合成输出张量。
            输出张量的形状为 (sequence_length, batch_size, emb_size)，即每个单词索引被映射到一个大小为 emb_size 的向量。
        '''

# Seq2Seq Network
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        '''
        num_encoder_layers = 3
        num_decoder_layers = 3
        emb_size = 512
        nhead = 8
        src_vocab_size = 19214
        tgt_vocab_size = 10837
        dim_feedforward = 512
        dropout = 0.1
        '''
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        '''
        1. d_model=emb_size:
            d_model 是 Transformer 模型中嵌入向量的维度。这里等于 emb_size，表示每个输入向量的维度。
            例如，如果 emb_size 是 512，那么每个输入向量的维度就是 512。
        2. nhead=nhead:
            nhead 是多头注意力机制中的头数。多头注意力机制通过并行计算多个注意力头，使模型能够关注输入的不同部分。
            例如，如果 nhead 是 8，那么每个注意力层会有 8 个并行的注意力头。
        3. num_encoder_layers=num_encoder_layers:
            num_encoder_layers 是编码器层的数量。每个编码器层包含一个多头注意力子层和一个前馈神经网络子层。
            例如，如果 num_encoder_layers 是 3，那么编码器会有 3 层。
        4. num_decoder_layers=num_decoder_layers:
            num_decoder_layers 是解码器层的数量。每个解码器层包含两个多头注意力子层和一个前馈神经网络子层。
            例如，如果 num_decoder_layers 是 3，那么解码器会有 3 层。
        5. dim_feedforward=dim_feedforward:
            dim_feedforward 是前馈神经网络层的维度。前馈神经网络层位于多头注意力子层之后，通常具有更高的维度。
            例如，如果 dim_feedforward 是 512，那么前馈神经网络的隐藏层维度就是 512。
        6. dropout=dropout:
            dropout 是 dropout 的比例，用于防止过拟合。它表示在每一层的训练过程中随机丢弃一部分神经元的概率。
            例如，如果 dropout 是 0.1，那么有 10% 的神经元会在训练过程中被随机丢弃。
        '''
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        # W: (tgt_vocab_size, emb_size)= (10837, 512)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        '''
            src shape torch.Size([27, 128])
            trg shape torch.Size([23, 128])
            src_mask shape torch.Size([27, 27])
            tgt_mask shape torch.Size([23, 23])
            src_padding_mask shape torch.Size([128, 27])
            tgt_padding_mask shape torch.Size([128, 23])
            memory_key_padding_mask shape torch.Size([128, 27])
            src_emb shape torch.Size([27, 128, 512])
            tgt_emb shape torch.Size([23, 128, 512])
            outs shape torch.Size([23, 128, 512])
            outs shape torch.Size([23, 128, 10837])
        
        src: (src_seq_len, batch_size)= (src_seq_len, 128)
        trg: (tgt_seq_len, batch_size)= (tgt_seq_len, 128)
        src_mask: (src_seq_len, src_seq_len)
        tgt_mask: (tgt_seq_len, tgt_seq_len)
        src_padding_mask: (batch_size, src_seq_len)= (128, src_seq_len)
        tgt_padding_mask: (batch_size, tgt_seq_len)= (128, tgt_seq_len)
        memory_key_padding_mask: (batch_size, src_seq_len)= (128, src_seq_len)
        
        '''
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        # src_emb: (src_seq_len, batch_size, emb_size)= (src_seq_len, 128, 512)
        # src: (src_seq_len, batch_size)= (src_seq_len, 128)
        # self.src_tok_emb(src): (src_seq_len, batch_size, emb_size)= (src_seq_len, 128, 512)
        # self.positional_encoding(self.src_tok_emb(src)): (src_seq_len, batch_size, emb_size)= (src_seq_len, 128, 512)
        
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        # tgt_emb: (tgt_seq_len, batch_size, emb_size)= (tgt_seq_len, 128, 512)
        # trg: (tgt_seq_len, batch_size)= (tgt_seq_len, 128)
        # self.tgt_tok_emb(trg): (tgt_seq_len, batch_size, emb_size)= (tgt_seq_len, 128, 512)
        # self.positional_encoding(self.tgt_tok_emb(trg)): (tgt_seq_len, batch_size, emb_size)= (tgt_seq_len, 128, 512)
        
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        # outs: (tgt_seq_len, batch_size, emb_size)= (tgt_seq_len, 128, 512)
        
        outs = self.generator(outs)
        # W: (tgt_vocab_size, emb_size)= (10837, 512)
        # outs = outs · W.T : (tgt_seq_len, batch_size, emb_size) · (emb_size, tgt_vocab_size)= (tgt_seq_len, batch_size, tgt_vocab_size)= (tgt_seq_len, 128, tgt_vocab_size)
        return outs
        # self.generator(outs): (tgt_seq_len, batch_size, tgt_vocab_size)= (tgt_seq_len, 128, tgt_vocab_size)

    def encode(self, src: Tensor, src_mask: Tensor):
        # src: (src_seq_len, batch_size)
        # src_mask: (src_seq_len, src_seq_len)
        outs = self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)
        # outs: (src_seq_len, batch_size, emb_size)= (src_seq_len, 128, 512)
        return outs

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        # tgt: (tgt_seq_len, batch_size)
        # memory: (src_seq_len, batch_size, emb_size)
        # tgt_mask: (tgt_seq_len, tgt_seq_len)
        outs = self.transformer.decoder(self.positional_encoding(
                            self.tgt_tok_emb(tgt)), memory,
                            tgt_mask)
        # outs: (tgt_seq_len, batch_size, emb_size)
        return outs

During training, we need a subsequent word mask that will prevent the
model from looking into the future words when making predictions. We
will also need masks to hide source and target padding tokens. Below,
let\'s define a function that will take care of both.

在训练期间，我们需要一个后续词掩码，以防止模型在进行预测时查看未来的词。我们还需要掩码来隐藏源和目标的填充标记。下面，让我们定义一个函数来处理这两个任务。

In [6]:
def generate_square_subsequent_mask(sz):
    # sz：掩码矩阵的大小，表示序列的长度
    # sz = src_seq_len or tgt_seq_len
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    # torch.ones((sz, sz), device=DEVICE): (sz, sz)
    # torch.triu(...): (sz, sz)
    # torch.triu(...).transpose(0, 1): (sz, sz)
    # mask: (sz, sz)
    '''
    1. 创建上三角矩阵:
        torch.ones((sz, sz), device=DEVICE) 创建一个大小为 (sz, sz) 的全 1 矩阵
        torch.triu(...) 生成上三角矩阵，即矩阵中所有位于主对角线及其上方的元素保持为 1，其余元素为 0
        == 1 将矩阵中的 1 转换为 True，0 转换为 False，生成布尔矩阵
    2. 转置矩阵:
        .transpose(0, 1) 对矩阵进行转置，交换矩阵的行和列
    '''
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    '''
    mask.float() 将布尔矩阵转换为浮点型矩阵
    .masked_fill(mask == 0, float('-inf')) 将矩阵中的 False（即原来的 0）位置填充为负无穷大 (-inf)
    .masked_fill(mask == 1, float(0.0)) 将矩阵中的 True（即原来的 1）位置填充为 0
    '''
    return mask
    '''
    假设sz = 5，则：
    torch.triu(torch.ones((5, 5), device=DEVICE)) == 1:
        tensor([[ True,  True,  True,  True,  True],
                [False,  True,  True,  True,  True],
                [False, False,  True,  True,  True],
                [False, False, False,  True,  True],
                [False, False, False, False,  True]], device='cuda:0')
    .transpose(0, 1)
        tensor([[ True, False, False, False, False],
                [ True,  True, False, False, False],
                [ True,  True,  True, False, False],
                [ True,  True,  True,  True, False],
                [ True,  True,  True,  True,  True]], device='cuda:0')
    mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        tensor([[ 0., -inf, -inf, -inf, -inf],
                [ 0.,  0., -inf, -inf, -inf],
                [ 0.,  0.,  0., -inf, -inf],
                [ 0.,  0.,  0.,  0., -inf],
                [ 0.,  0.,  0.,  0.,  0.]], device='cuda:0')
    '''

def create_mask(src, tgt):
    '''
    src or tgt = 
    tensor([[ 2,  2,  2,  ...,  2,  2,  2],
            [21, 84,  5,  ..., 21, 14, 14],
            [85, 31, 69,  ..., 46, 38, 17],
            ...,
            [ 1,  1,  1,  ...,  1,  1,  1],
            [ 1,  1,  1,  ...,  1,  1,  1],
            [ 1,  1,  1,  ...,  1,  1,  1]], device='cuda:0')
    '''
    # src: (src_seq_len, batch_size)
    # tgt: (tgt_seq_len - 1, batch_size)
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    # tgt_mask: (tgt_seq_len - 1, tgt_seq_len - 1)
    '''
    tensor([[ 0., -inf, -inf, ..., -inf],
            [ 0.,  0., -inf, ..., -inf],
            ...
            [ 0.,  0.,  ...,  0., -inf],
            [ 0.,  0.,  ...,  0.,  0.]], device='cuda:0')
    '''
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)
    # src_mask: (src_seq_len, src_seq_len)
    '''
    tensor([[False, ..., False,
            ...,
            [False, ..., False]], device='cuda:0')
    '''

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    # src_padding_mask: (batch_size, src_seq_len)
    '''
    src == PAD_IDX: 生成一个布尔张量，形状与 src 相同。布尔张量中每个元素表示 src 中对应位置的元素是否等于 PAD_IDX。如果等于 PAD_IDX，则该位置的值为 True，否则为 False
    .transpose(0, 1): 将生成的布尔张量进行转置，即交换其第 0 维和第 1 维。转置后，张量的形状将变为 (batch_size, sequence_length)。转置的原因是为了符合模型中某些操作的输入需求
    '''

    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    # tgt_padding_mask: (batch_size, tgt_seq_len - 1)
    '''
    tensor([[False, False, False,  ...,  True,  True,  True],
            [False, False, False,  ...,  True,  True,  True],
            [False, False, False,  ...,  True,  True,  True],
            ...,
            [False, False, False,  ...,  True,  True,  True],
            [False, False, False,  ...,  True,  True,  True],
            [False, False, False,  ...,  True,  True,  True]], device='cuda:0')
    '''
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [7]:
import torch

# 创建一个形状为 (10, 32) 的张量
tensor = torch.randn(10, 32)

# 使用 size 方法获取张量在第一个维度的大小
first_dim_size = tensor.size(0)
print(f"First dimension size using size(0): {first_dim_size}")
shape = tensor.shape
size = tensor.size()

print(f"Shape: {shape}")
print(f"Size: {size}")
print(tensor.size(1))
print(size[1]) # print(size(1)) 报错
print(shape[1])

First dimension size using size(0): 10
Shape: torch.Size([10, 32])
Size: torch.Size([10, 32])
32
32
32


Let\'s now define the parameters of our model and instantiate the same.
Below, we also define our loss function which is the cross-entropy loss
and the optimizer used for training.

现在让我们定义模型的参数并实例化模型。下面，我们还定义了我们的损失函数，即交叉熵损失，以及用于训练的优化器。

In [8]:
torch.manual_seed(0)

SRC_VOCAB_SIZE = len(vocab_transform[SRC_LANGUAGE])
# SRC_VOCAB_SIZE = 19214
TGT_VOCAB_SIZE = len(vocab_transform[TGT_LANGUAGE])
# TGT_VOCAB_SIZE = 10837
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 128
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3

transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)

for p in transformer.parameters():
    # transformer.parameters()：获取模型 transformer 的所有参数
    if p.dim() > 1:
        # 检查参数 p 的维度，如果维度大于 1，则对其进行初始化。通常，维度大于 1 的参数是权重矩阵（例如，线性层或卷积层的权重），而维度为 1 的参数通常是偏置项
        nn.init.xavier_uniform_(p)
        # 使用 Xavier uniform 方法初始化参数 p
        '''
        Xavier 初始化（也称为 Glorot 初始化）是一种权重初始化方法，旨在使前向传播过程中每层的输出方差相同。它可以有效地避免梯度消失和梯度爆炸问题，特别是在深层网络中。Xavier uniform 初始化方法将权重初始化为从一个均匀分布中抽取的值，其范围由输入和输出的节点数量决定。
        由 Xavier Glorot 和 Yoshua Bengio 在论文 "Understanding the difficulty of training deep feedforward neural networks" 中提出
        '''
        

transformer = transformer.to(DEVICE)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)
'''
为什么使用 ignore_index: 在处理序列数据（如自然语言处理任务）时，序列通常具有不同的长度。为了使这些序列能够组成一个批次，较短的序列需要填充到相同的长度。填充值在训练过程中不应该对损失计算产生影响，因此使用 ignore_index 参数来忽略这些填充值，可以确保模型只关注实际数据部分，避免填充值对模型训练的干扰。
'''

optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)


'''
警告信息指出 enable_nested_tensor 设置为 True，但 self.use_nested_tensor 设置为 False，这是因为 encoder_layer.self_attn.batch_first 并未设置为 True。以下是详细解释：
    enable_nested_tensor：这是一个用于启用嵌套张量（Nested Tensor）功能的标志。嵌套张量可以在某些情况下提高性能。
    self.use_nested_tensor：这是一个内部变量，指示是否实际使用嵌套张量。
    encoder_layer.self_attn.batch_first：这是 Transformer 模型中自注意力层的配置，指示输入张量的批次维度是否在第一位。如果设置为 True，则输入张量的形状通常是 (batch_size, seq_length, embedding_dim)。
'''

/data/xuruiyuan/miniconda3/lib/python3.12/site-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


'\n警告信息指出 enable_nested_tensor 设置为 True，但 self.use_nested_tensor 设置为 False，这是因为 encoder_layer.self_attn.batch_first 并未设置为 True。以下是详细解释：\n    enable_nested_tensor：这是一个用于启用嵌套张量（Nested Tensor）功能的标志。嵌套张量可以在某些情况下提高性能。\n    self.use_nested_tensor：这是一个内部变量，指示是否实际使用嵌套张量。\n    encoder_layer.self_attn.batch_first：这是 Transformer 模型中自注意力层的配置，指示输入张量的批次维度是否在第一位。如果设置为 True，则输入张量的形状通常是 (batch_size, seq_length, embedding_dim)。\n'

Collation 整理
=========

As seen in the `Data Sourcing and Processing` section, our data iterator
yields a pair of raw strings. We need to convert these string pairs into
the batched tensors that can be processed by our `Seq2Seq` network
defined previously. Below we define our collate function that converts a
batch of raw strings into batch tensors that can be fed directly into
our model.

如在 `数据获取和处理` 部分所见，我们的数据迭代器生成一对原始字符串。我们需要将这些字符串对转换为可以由我们之前定义的 `Seq2Seq` 网络处理的批量张量。下面我们定义我们的整理函数，该函数将一批原始字符串转换为可以直接输入到我们模型中的批量张量。

In [9]:
from torch.nn.utils.rnn import pad_sequence

# helper function to club together sequential operations
# 辅助函数，用于将一系列操作组合在一起
def sequential_transforms(*transforms):
    '''
    将多个变换函数按顺序应用于输入数据。具体来说，sequential_transforms 接受任意数量的变换函数，并返回一个新的函数 func，该函数会将输入数据依次通过这些变换函数进行处理
    *transforms 表示 sequential_transforms 函数可以接受任意数量的变换函数作为参数。这些变换函数会被传递为一个元组
    '''
    def func(txt_input):
        '''
        func 是一个内部函数，它接受一个输入 txt_input。
        for transform in transforms: 循环遍历传递给 sequential_transforms 的所有变换函数。
        txt_input = transform(txt_input) 将输入数据依次通过每个变换函数进行处理，并更新 txt_input。
        最后，func 返回经过所有变换函数处理后的数据
        '''
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func
    # sequential_transforms 返回内部定义的 func 函数。这个函数会按顺序应用所有传递的变换函数

# function to add BOS/EOS and create tensor for input sequence indices
# 添加 BOS/EOS 并为输入序列索引创建张量的函数
def tensor_transform(token_ids: List[int]):
    '''
    将一个包含标记 ID 的列表转换为一个 PyTorch 张量，并在该张量的开头和结尾分别添加起始标记（BOS, Begin Of Sequence）和结束标记（EOS, End Of Sequence）
    '''
    return torch.cat((torch.tensor([BOS_IDX]),
                      torch.tensor(token_ids),
                      torch.tensor([EOS_IDX])))
    # (len(token_ids) + 2,)

# ``src`` and ``tgt`` language text transforms to convert raw strings into tensors indices
# 将原始字符串转换为张量索引的 `src` 和 `tgt` 语言文本转换
text_transform = {}
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    text_transform[ln] = sequential_transforms(token_transform[ln], #Tokenization
                                               vocab_transform[ln], #Numericalization
                                               tensor_transform) # Add BOS/EOS and create tensor
    '''
    sequential_transforms 函数将多个变换函数组合在一起，按顺序应用于输入数据
    token_transform[ln] 是第一个变换函数，负责将输入文本进行分词（Tokenization）。
    vocab_transform[ln] 是第二个变换函数，负责将分词后的文本进行数值化（Numericalization），即将每个单词映射为对应的索引。
    tensor_transform 是第三个变换函数，负责在数值化后的标记序列中添加起始（BOS）和结束（EOS）标记，并将其转换为 PyTorch 张量。
    这些变换函数组合成一个新的函数，并存储在 text_transform 字典中，键为语言标识符 ln。
    '''
    '''
    'Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.'
    ['Zwei', 'junge', 'weiße', 'Männer', 'sind', 'im', 'Freien', 'in', 'der', 'Nähe', 'vieler', 'Büsche', '.']
    [21, 85, 257, 31, 87, 22, 94, 7, 16, 112, 7910, 3209, 4]
    tensor([   2,   21,   85,  257,   31,   87,   22,   94,    7,   16,  112, 7910,        3209,    4,    3])
    '''


# function to collate data samples into batch tensors
# 将数据样本整理为批量张量的函数
def collate_fn(batch):
    # len(batch) = batch_size = 128
    # batch 是一个list，每个元素是一个元组 (src_sample, tgt_sample)，分别表示源语言和目标语言的样本
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        # 遍历批次中的每个样本，将源语言和目标语言的样本分别添加到 src_batch 和 tgt_batch 列表中
        src_batch.append(text_transform[SRC_LANGUAGE](src_sample.rstrip("\n")))
        # rstrip("\n") 用于去除样本末尾的换行符
        tgt_batch.append(text_transform[TGT_LANGUAGE](tgt_sample.rstrip("\n")))

    # src_batch: [(src_seq_len_1,), (src_seq_len_2,), ..., (batch_size,)]
    # tgt_batch: [(tgt_seq_len_1,), (tgt_seq_len_2,), ..., (batch_size,)]
    # pad_sequence 函数会将输入的列表中的每个张量填充到相同的长度，以便可以形成一个统一的批次张量
    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch

Let\'s define training and evaluation loop that will be called for each
epoch.

定义一个训练和评估循环，该循环将在每个 epoch 中调用。

In [10]:
from torch.utils.data import DataLoader

def train_epoch(model, optimizer):
    model.train()
    losses = 0
    train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    train_dataloader = DataLoader(train_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)

    for src, tgt in train_dataloader:
        # src: (src_seq_len, batch_size)= (src_seq_len, 128)
        # tgt: (tgt_seq_len, batch_size)= (tgt_seq_len, 128)
        '''
        目标序列输入 tgt_input 用于模型的输入，帮助模型在每个时间步预测下一个标记。
        目标序列输出 tgt_out 用于与模型的预测结果进行比较，计算损失
        tgt序列长度为n，则前n-1个标记作为输入，后n-1个标记作为输出label
        tgt = torch.tensor([[1, 2],
                            [3, 4],
                            [5, 6],
                            [7, 8],
                            [9, 10]])
        tgt_input = tgt[:-1, :] =   tensor([[1, 2],
                                            [3, 4],
                                            [5, 6],
                                            [7, 8]])
        tgt_out = tgt[1:, :] =  tensor([[ 3,  4],
                                        [ 5,  6],
                                        [ 7,  8],
                                        [ 9, 10]])
        '''
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]
        # tgt_input: (tgt_seq_len - 1, batch_size)= (tgt_seq_len - 1, 128)

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)
        # src_mask: (src_seq_len, src_seq_len)
        # tgt_mask: (tgt_seq_len - 1, tgt_seq_len - 1)
        # src_padding_mask: (batch_size, src_seq_len)= (128, src_seq_len)
        # tgt_padding_mask: (batch_size, tgt_seq_len - 1)= (128, tgt_seq_len - 1)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)
        # logits: (tgt_seq_len - 1, batch_size, tgt_vocab_size)= (tgt_seq_len - 1, 128, 10837)
        # model内的维度懒得改了, tgt_seq_len 应该是 tgt_seq_len - 1

        optimizer.zero_grad()

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        # logits.reshape(-1, logits.shape[-1]): ((tgt_seq_len - 1) * batch_size, tgt_vocab_size)= ((tgt_seq_len - 1) * 128, 10837)
        # tgt_out.reshape(-1): ((tgt_seq_len - 1) * batch_size)= ((tgt_seq_len - 1) * 128)
        '''
        logits:
            模型的预测结果，形状为 (tgt_seq_len - 1, batch_size, vocab_size)
            tgt_seq_len - 1 是目标序列的长度减去 1（因为 tgt_input 去掉了最后一个标记）
        logits.reshape(-1, logits.shape[-1]):
            logits.reshape(-1, logits.shape[-1]) 将 logits 张量重新形状化为 (N, vocab_size)，其中 N 是 (tgt_seq_len - 1) * batch_size。
            这样做是为了将预测的结果展平成二维张量，每一行表示一个时间步的预测结果
        tgt_out:
            tgt_out 是目标序列输出，形状为 (tgt_seq_len - 1, batch_size)
        tgt_out.reshape(-1):
            tgt_out.reshape(-1) 将 tgt_out 张量展平成一维张量，形状为 (N,)，其中 N 是 (tgt_seq_len - 1) * batch_size。
            这样做是为了将目标输出展平成一维张量，每个元素对应一个时间步的真实标记
        '''
        loss.backward()

        optimizer.step()
        losses += loss.item()

    return losses / len(list(train_dataloader))


def evaluate(model):
    model.eval()
    losses = 0

    val_iter = Multi30k(split='valid', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    val_dataloader = DataLoader(val_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)

    for src, tgt in val_dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()

    return losses / len(list(val_dataloader))

Now we have all the ingredients to train our model. Let\'s do it!

现在我们已经拥有了训练模型所需的所有要素。让我们开始吧！

In [11]:
from timeit import default_timer as timer
NUM_EPOCHS = 18

for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss = train_epoch(transformer, optimizer)
    end_time = timer()
    val_loss = evaluate(transformer)
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))


# function to generate output sequence using greedy algorithm
# 生成输出序列的贪婪算法函数
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    # src: (src_seq_len, infer_batch_size)= (src_seq_len, 1)
    
    src_mask = src_mask.to(DEVICE)
    # src_mask: (src_seq_len, src_seq_len)= (src_seq_len, src_seq_len)
    # 全False

    # 编码器的输出
    memory = model.encode(src, src_mask)
    # memory: (src_seq_len, infer_batch_size, emb_size)= (src_seq_len, 1, 512)

    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    # ys: (current_seq_len, infer_batch_size)= (1, 1)
    '''
    1. 创建全 1 张量:
        torch.ones(1, 1) 创建一个形状为 (1, 1) 的全 1 张量
        这个张量表示一个批次大小为 1 的序列，其中只有一个时间步
    2. 填充起始标记:
        .fill_(start_symbol) 将张量的所有元素填充为 start_symbol
        start_symbol 是起始标记的索引（例如 <bos> 的索引）
    3. 转换数据类型:
        .type(torch.long) 将张量的元素数据类型转换为 long 类型，适合表示词汇表中的索引
    4. 移动到设备:
        .to(DEVICE) 将张量移动到指定的设备上
    ys = tensor([[2]], device='cuda:0')
    贪心解码（greedy decoding）过程中，ys 作为目标序列的初始输入，包含起始标记 BOS_IDX。解码器将根据这个初始输入生成序列的下一个标记，并逐步扩展目标序列，直到达到最大长度或遇到结束标记 EOS_IDX
    ys 从包含起始标记的初始张量开始，不断扩展，添加新的预测标记，直到达到指定的最大长度 max_len 或遇到结束标记 EOS_IDX。通过这种方式，模型可以逐步生成整个目标序列。
    '''
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        # memory: (src_seq_len, infer_batch_size, emb_size)= (src_seq_len, 1, 512)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(DEVICE)
        # tgt_mask: (current_seq_len, current_seq_len)= (i + 1, i + 1)
        '''
        1. 获取目标序列长度:
            ys 是目标序列张量，形状为 (current_seq_len, 1)，其中 current_seq_len 是当前目标序列的长度
            ys.size(0) 获取目标序列的长度 current_seq_len
        2. 生成后续掩码:
            generate_square_subsequent_mask 是一个函数，用于生成方形的后续掩码，形状为 (current_seq_len, current_seq_len)
            这个掩码用于在自注意力机制中屏蔽未来的时间步
        3. 转换为布尔类型:
            .type(torch.bool) 将生成的掩码矩阵转换为布尔类型。
            在布尔类型中，0 被视为 False，1 被视为 True
        4. 移动到设备
        '''
        out = model.decode(ys, memory, tgt_mask)
        # out: (current_seq_len, infer_batch_size, emb_size)= (i + 1, 1, 512)
        
        out = out.transpose(0, 1)
        # out: (infer_batch_size, current_seq_len, emb_size)= (1, i + 1, 512)
        # 在 Transformer 解码器的输出中，张量的形状通常为 (current_seq_len, batch_size, d_model)。然而，在许多情况下，处理批次数据时更方便的张量形状是 (batch_size, current_seq_len, d_model)。因此，通过转置操作可以更方便地进行后续处理，例如计算损失或进行进一步的生成步骤。
        
        # 通过模型的生成器层将解码器的输出转换为预测的概率分布
        prob = model.generator(out[:, -1])
        # out[:, -1]: (infer_batch_size, emb_size)= (1, 512)
        # prob: (infer_batch_size, vocab_size)= (1, 10837)
        
        '''
        1. 提取最后一个时间步的输出:
            out 是解码器的输出张量，形状为 (infer_batch_size, current_seq_len, emb_size)。
            out[:, -1] 提取每个序列的最后一个时间步的输出，形状为 (infer_batch_size, emb_size)。
        2. 通过生成器层:
            model.generator 是一个线性层（通常是 nn.Linear），用于将解码器的输出维度 emb_size 映射到词汇表大小 vocab_size。
            通过生成器层后，prob 的形状为 (infer_batch_size, vocab_size)，表示每个时间步预测的词汇表中每个单词的概率分布
        
        out[:, -1] 使用切片操作，从张量 out 中提取每个序列的最后一个时间步的输出。
        具体来说，: 表示选择所有批次，-1 表示选择最后一个时间步。
        结果是一个形状为 (infer_batch_size, emb_size) 的张量，其中每个元素是对应批次的最后一个时间步的输出向量
        其他例子: 
        解码器输出张量 (out):
        tensor([[[ 0.4346,  0.7125, -0.7205,  ...,  0.3390, -1.0318,  1.0596],
                [-0.4125,  0.5001,  0.5412,  ...,  1.2661, -1.0583, -0.1175],
                [ 0.5393, -1.3724, -0.1473,  ...,  0.6777, -0.3928,  1.0025],
                [ 1.0424, -0.6054, -1.3143,  ...,  0.5600,  1.1926, -0.3984],
                [ 0.6790,  1.2351, -0.2945,  ..., -1.0287,  0.1314, -1.0034]],

                [[-0.6451, -0.2518,  0.8535,  ...,  1.0723, -0.5115,  0.6872],
                [-0.6250, -1.4958,  1.0008,  ..., -0.2207,  1.1428, -1.0486],
                [ 0.4608, -0.2767, -0.2893,  ..., -1.3023, -0.2460,  0.5015],
                [ 1.1483,  0.1402, -0.5167,  ...,  1.1991,  1.1763, -0.3072],
                [-0.4706, -0.5463, -1.1617,  ..., -0.4990, -0.4731,  1.1536]]])
        张量形状: torch.Size([2, 5, 512])

        最后一个时间步的输出 (last_timestep_output):
        tensor([[ 0.6790,  1.2351, -0.2945,  ..., -1.0287,  0.1314, -1.0034],
                [-0.4706, -0.5463, -1.1617,  ..., -0.4990, -0.4731,  1.1536]])
        张量形状: torch.Size([2, 512])
        '''
        
        # prob: (infer_batch_size, vocab_size)= (1, 10837)
        _, next_word = torch.max(prob, dim=1)
        # next_word: (infer_batch_size,)= (1,)
        '''
        输入张量 prob:
            prob 是通过生成器层得到的概率分布，形状为 (infer_batch_size, vocab_size)。
            infer_batch_size 是批次大小，vocab_size 是词汇表的大小。
            prob 的每一行是一个长度为 vocab_size 的向量，表示每个单词在当前时间步的预测概率
        获取最大概率及其索引:
            torch.max 函数用于获取张量中指定维度上的最大值和相应的索引。
            dim=1 指定在第 1 维度（从 0 开始计数）上找到最大值，dim=1 表示在词汇表维度上进行操作。
            返回两个张量：最大值和最大值的索引。
                最大值表示最高的预测概率（不在此处使用）。
                最大值的索引表示具有最高概率的单词的索引。
        忽略最大值，只获取索引:
            _ 用于忽略 torch.max 返回的第一个张量（最大值）。
            next_word 保存第二个张量（最大值的索引），形状为 (infer_batch_size,)。
        '''
        # 将 next_word 张量中的值提取出来，并将其转换为一个 Python 标量
        next_word = next_word.item()

        # ys: (current_seq_len, infer_batch_size)= (1, 1)
        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        '''
        1. 创建一个新的张量: torch.ones(1, 1).type_as(src.data).fill_(next_word)
            torch.ones(1, 1) 创建一个形状为 (1, 1) 的全 1 张量, (1, infer_batch_size)
            .type_as(src.data) 将这个张量的数据类型设置为与 src.data 相同的类型。
            .fill_(next_word) 用 next_word 的值填充这个张量。
            结果是一个形状为 (1, 1) 的张量，值为 next_word。
        2. 扩展目标序列 ys: torch.cat([ys, torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
            torch.cat 函数用于沿指定维度连接张量。
            [ys, torch.ones(1, 1).type_as(src.data).fill_(next_word)] 创建一个包含当前目标序列 ys 和新创建的张量的列表。
            dim=0 指定在第 0 维度（行）上进行连接。
            结果是一个新的张量 ys，形状为 (current_seq_len + 1, infer_batch_size)= (i + 2, 1)
        举例:
        当前目标序列 ys:
            tensor([[1],
                    [2],
                    [3]])
        扩展后的目标序列 ys:
            tensor([[1],
                    [2],
                    [3],
                    [4]])
        '''
        if next_word == EOS_IDX:
            break
    return ys


# actual function to translate input sentence into target language
def translate(model: torch.nn.Module, src_sentence: str):
    model.eval()
    src = text_transform[SRC_LANGUAGE](src_sentence).view(-1, 1)
    # text_transform[SRC_LANGUAGE](src_sentence): (src_seq_len,)
    # src = text_transform[SRC_LANGUAGE](src_sentence).view(-1, 1): (src_seq_len, 1)
    '''
    1. 文本转换:
        text_transform[SRC_LANGUAGE] 是一个为源语言定义的文本转换函数。
        src_sentence 是输入的源语言句子，它是一个字符串。
        text_transform[SRC_LANGUAGE](src_sentence) 将输入的源语言句子进行一系列预处理，包括分词（Tokenization）、数值化（Numericalization）和添加起始/结束标记，并最终转换为 PyTorch 张量
    2. 视图转换:
        .view(-1, 1) 将转换后的张量重新形状化为 (src_seq_len, 1)。
        其中 src_seq_len 是输入句子的长度（包括起始和结束标记），1 表示批次大小为 1。
    '''
    num_tokens = src.shape[0]
    # num_tokens = src_seq_len
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    # src_mask: (src_seq_len, src_seq_len)
    tgt_tokens = greedy_decode(
        model, src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
    return " ".join(vocab_transform[TGT_LANGUAGE].lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")

/data/xuruiyuan/miniconda3/lib/python3.12/site-packages/torch/nn/functional.py:5137: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
/data/xuruiyuan/miniconda3/lib/python3.12/site-packages/torch/utils/data/datapipes/iter/combining.py:337: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read from the start again.
  warnings.warn("Some child DataPipes are not exhausted when __iter__ is called. We are resetting "


Epoch: 1, Train loss: 5.344, Val loss: 4.106, Epoch time = 16.017s
Epoch: 2, Train loss: 3.763, Val loss: 3.319, Epoch time = 14.629s


In [12]:
print(translate(transformer, "Eine Gruppe von Menschen steht vor einem Iglu ."))

 A group of people are standing in front of a crowd . 


References 参考文献
==========

1.  Attention is all you need paper.
    <https://papers.nips.cc/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf>
2.  The annotated transformer. 注释版transformer
    <https://nlp.seas.harvard.edu/2018/04/03/attention.html#positional-encoding>